# Experiment 1 - Zero-shot intent inference
Infer intent using zero-shot by providing the intent questions in the system prompt

## Setup, imports, logging, keys

In [49]:
import os
from dotenv import load_dotenv, find_dotenv
import openai
import pandas as pd
import ipywidgets as widgets
import time
import logging

import logging
logger = logging.getLogger()
fhandler = logging.FileHandler(filename='mylog.log', mode='a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.DEBUG)

In [50]:

_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key  = os.environ['OPENAI_API_KEY']

## Step 1 - System prompt to infer intent question 
We have n number of questions, each represents an intent. Our system prompt instructs gpt how to take an original user question and attempt to inferthe intent with a zero shot approach. The prompt also tells gpt what to do if it does not know how to do the infrenece. It tries to guardrail the prompt a bit so that gpt does not get creative and add things to the response. 


In [51]:
system_prompt =  """
You are a helpful assistant for the healthcare industry, you understand the administrative \
aspects of healthcare and also have a clinical background. You understand \
pharmacy terminology and prescription dosages. 

You will infer the intent question from the user prompt, use the following list of intent questions below \
as reference. Your task is to repeat back the exact same inferred intent question as written originally. \
If you can't, then \
respond "please reframe your question." Do not take parts of the user message and add to your response. Only \
provide one answer, do not combine two. 
-------
Are there limitations that apply to my drug, such as prior authorization?
Can I sign up for paperless delivery of my health plan documents and communications?
How can I cancel my current insurance plan?
How can I find a network PCP, specialist, clinic or hospital?
How can I replace a lost card or order additional insurance cards?
How do I file a complaint?
How do I know if a generic version of the medication is available?
How do I request pre-authorization?
How do I set up a virtual visit (telehealth visit)?
How much can I earn for reward?
I am a new member and just received my member ID card. What should I do now?
I don't want to talk
I need to search for my doctors in another language. How can I do that?
I want to appeal a coverage decision that was denied. How do I do that?
I want to ask for coverage for a medical service or prescription drug. How do I request a coverage decision?
I'd like to speak to a licensed agent
What are the rewards, and how can I use it?
What do I need to do to earn reward?
What extra benefits does my plan offer?
What if I need help managing my medications?
What is digital health concierge?
What is member onboarding process?
What is this about?
What services are covered by my health plan and what I am responsible for paying?
What's my plan coverage for ambulance?
What's my plan coverage for ambulatory surgical center?
What's my plan coverage for annual physical?
What's my plan coverage for blood glucose test strips?
What's my plan coverage for blood test?
What's my plan coverage for cancer screening?
What's my plan coverage for cardiologist?
What's my plan coverage for chiropractic care?
What's my plan coverage for CT?
What's my plan coverage for dental cleaning?
What's my plan coverage for dental crowns?
What's my plan coverage for dental service?
What's my plan coverage for dentures and dental implants?
What's my plan coverage for depression screenings?
What's my plan coverage for diabetic monitoring supplies?
What's my plan coverage for diagnostic radiology?
What's my plan coverage for durable medical equipment?
What's my plan coverage for emergency & urgently needed care worldwide?
What's my plan coverage for emergency care in the united states?
What's my plan coverage for eyewear coverage?
What's my plan coverage for fitness?
What's my plan coverage for glucose monitors?
What's my plan coverage for ground ambulance?
What's my plan coverage for hearing?
What's my plan coverage for in and out of network doctor visits?
What's my plan coverage for injectible?
What's my plan coverage for inpatient admission?
What's my plan coverage for inpatient surgery?
What's my plan coverage for lab services?
What's my plan coverage for meals?
What's my plan coverage for medicare part B drugs?
What's my plan coverage for my drug?
What's my plan coverage for nebulizers?
What's my plan coverage for nurse line?
What's my plan coverage for oral exam?
What's my plan coverage for outpatient service?
What's my plan coverage for outpatient X-ray?
What's my plan coverage for over the counter drug?
What's my plan coverage for preventive benefits?
What's my plan coverage for primary care physician?
What's my plan coverage for prosthetics (e.g. braces, artificial limbs)?
What's my plan coverage for root canals?
What's my plan coverage for specialist?
What's my plan coverage for telehealth?
What's my plan coverage for transplants?
What's my plan coverage for urgently needed care in the united states?
What's my plan coverage for vaccines and flu shots?
What's my plan coverage for vision?
What's my plan coverage for wheelchair?
What's my plan coverage when I'm outside of the plan's service area?
What's my plan deductible?
What's my plan premium?
What's my plan's maximum out of pocket?
"""

## Step 2 - Try the prompt with a couple examples
Try this user prompt for which we expect to be able to classify as the following intent question: 

What's my plan coverage for over the counter drug?

In [52]:
user_prompt = "Can I get OTC nicotine patches, antacids, and cold medicine with my plan's OTC allowance?"


Let's add a function that will get a response from gpt given the promots and can also do exponential backoff if we get a timeout, that will come in handy for when we run multiple requests from the csv below. 

In [57]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, 
                                 top_p = 0,
                                 max_tokens=500):
    retry_count = 0
    max_retries = 4
    wait_time = 7  # Initial wait time in seconds

    while retry_count < max_retries:
        try:

            user_content = next((message['content'] for message in messages if message['role'] == 'user'), None)

            logger.debug(f"Trye the following message: {user_content}")
            response = openai.ChatCompletion.create(
                model=model,
                messages=messages,
                temperature=temperature,
                max_tokens=max_tokens,
            )
            
            # print(completion['usage']['total_tokens'])
            return response.choices[0].message["content"]

        except Exception as e:
            print(f"An error occurred: {str(e)[:30]}")
            print(f"Retrying in {wait_time} seconds...")

            logger.debug(f"An error occurred: {str(e)[:30]}")
            logger.debug(f"Retrying in {wait_time} seconds...")
            
            time.sleep(wait_time)
            retry_count += 1
            wait_time *= 3  # Exponential backoff: double the wait time for each retry

    print("Exceeded maximum number of retries. Aborting.")
    return None



Let's combine the system and user prompt and try to get a response

In [58]:
messages =  [  
{'role':'system', 'content': system_prompt},    
{'role':'user', 'content': user_prompt},  
] 
response = get_completion_from_messages(messages)
print(response)

What's my plan coverage for over the counter drug?


---------
Let's try another question from the same category

In [56]:
user_prompt = "I'm interested in utilizing my OTC benefit - can I use it for things like cough drops, aspirin, and antihistamines?"

messages =  [  
{'role':'system', 'content': system_prompt},    
{'role':'user', 'content': user_prompt},  
] 
response = get_completion_from_messages(messages)
print(response)


What's my plan coverage for over the counter drug?


---

## Step 3 - Test system prompt across all examples

We have a file named **bot_question_intent_map.csv**, it has a list of example user questions with its respective categorical intent question. We will load the whole file and for each pair see what intent question our system prompt predicts.



In [43]:
df = pd.read_csv('bot_question_intent_map.csv')

In [44]:
total_rows = len(df)

In [45]:
def inferred_intent(system, question, expected, total, current):
    messages =  [  
    {'role':'system', 'content': system},    
    {'role':'user', 'content': question},  
    ] 


    response = get_completion_from_messages(messages)
    
    return response

In [46]:
df['inferred'] = df.apply(lambda row: inferred_intent(system_prompt, row['question'], row['intent question'], total_rows, row.index), axis=1)


An error occurred: Request timed out: HTTPSConnec
Retrying in 7 seconds...
An error occurred: Request timed out: HTTPSConnec
Retrying in 7 seconds...


In [47]:
df.to_csv('bot_inferred_intent_gpt.csv', index=False)